
# Introducing Supriya

A Python API for SuperCollider

https://github.com/josiah-wolf-oberholtzer/supriya

## Supriya lets you...

- boot and communicate with SuperCollider's `scsynth` synthesis server

- construct and compile SynthDef unit generator graphs in native Python code

- build and control graphs of synthesizers and synthesizer groups

- explicitly object-model `scsynth`-specific OSC commands  via `Request` and `Response` classes

- compile non-realtime synthesis scores via Supriya's `Session` class

- write patterns for realtime or non-realtime synthesis

A lot of the same stuff you do with `sclang` and `scide`, just in Python instead.

## About the author

- A composer and programmer
  - https://github.com/josiah-wolf-oberholtzer
  - https://soundcloud.com/josiah-wolf-oberholtzer/in-the-tall-grasses

- PhD from Harvard in Music Composition, specializing in massively multi-channel tape music and symbolic computer-assisted composition

- Core contributor to Abjad (https://http://abjad.mbrsi.org/), a Python API for LilyPond

- Software engineering manager at Capital One, running a group developing serverless machine learning applications for hotel reservation arbitrage

- Used / taught enough Max/MSP to hit a wall

## Hold up, what's Python? (Cribbed from Wikipedia)

- Python is an interpreted, high-level, general-purpose programming language.

- Created by Guido van Rossum and first released in 1991, **Python has a design philosophy that emphasizes code readability, notably using significant whitespace.** (emphasis mine)

- It provides constructs that enable clear programming on both small and large scales.

- Python features a dynamic type system and automatic memory management.

- It supports multiple programming paradigms, including object-oriented, imperative, functional and procedural, and has a large and comprehensive standard library. It also has a vibrant third-party package ecosystem.

## Hello Python

This is maybe a little corny, but it's pretty good advice.

In [1]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


## SuperCollider Architecture Review

- `scsynth`: a synthesis server
- `sclang`: a programming language / interpreter
- `scide`: a GUI for running all of the above
- `OSC (Open Sound Control)`: "a protocol for communication among computers, sound synthesizers, and other multimedia devices"

<img src="attachment:scEn.png" align="center" />

## OK, but why make another `scsynth` client?

- To take advantage of language features and libraries not available in `sclang`
- To explore language features and libraries in your chosen language you might not otherwise interact with
- To better understand how `sclang` and `scsynth` interact
- Just For Fun™
- Because you're stubborn

## No, really, why?

Well, I want...

- To make massively multichannel fixed media pieces in my preferred language, so I can make use of code I've already written as well as many third-party libraries I love using for documentation, testing, etc.
- To have all audio materials modeled in code
- To create parity between the experiences of realtime experimentation and non-realtime composing
- To allow layers of NRT material to reference one another in an object-oriented way
- To allow for fully-reproducible (re-)rendering of NRT scores
- To allow entire scores and related CLI/TUI tools to be fully tested as code

## Some design principles

- Keep the scope narrow / don't reinvent
    - Lean on the wider community for unit test, math, MIDI, etc.
- Keep the interfaces familiar
    - Python makes it easy for classes to implement list-like / dictionary-like interfaces
- Keep the interfaces simple
    - Avoid convenience methods, alternate spellings
- Avoid global state
- Make classes as immutable as possible
    - Local state can be as hard to manage as global state
    - Use factory classes to configure and instantiate instances
- If it's a term-of-art in `scsynth`, use the name

## Hello World

In [4]:
from supriya import Server, Synth

In [5]:
server = Server()
server.boot()

AttributeError: module 'os' has no attribute 'getpgid'

In [ ]:
synth = Synth()
synth.allocate()

In [ ]:
print(server)

In [ ]:
synth.release()

In [ ]:
server.quit()

### Hello World (a little more complicated)

In [ ]:
from supriya import Bus, Group, Server, Synth

In [ ]:
server = Server().boot()

In [ ]:
bus = Bus.control().allocate()
bus.set(0.5)

In [ ]:
group = Group().allocate()
for i in range(1, 10):
    synth = Synth(amplitude=bus, frequency=111 * i)
    _ = synth.allocate(target_node=group)

In [ ]:
bus.set(0.1)

In [ ]:
group.controls["gate"] = 0

In [ ]:
server.quit()

## Realtime Server Node Tree Model

Let's just import `supriya`:

In [ ]:
import supriya

And let's make a bunch of synths and groups...

In [ ]:
synth_a = supriya.Synth(frequency=333, name="A")
synth_b = supriya.Synth(frequency=444, name="B")
synth_c = supriya.Synth(frequency=555, name="C")
outer_group = supriya.Group(name="Outer")
inner_group = supriya.Group(name="Inner")
outer_group.extend([synth_a, inner_group, synth_c])
inner_group.append(synth_b)

Shhh, this is magic...

In [ ]:
%load_ext supriya.ext.ipython

...so we can capture some of Supriya's output in this presentation:

In [ ]:
_ = supriya.graph(outer_group)

Groups know about their children and nodes know about their parents:

In [ ]:
for index, node in enumerate(outer_group.children):
    print(index, repr(node)) 

In [ ]:
for distance, node in enumerate(synth_b.parentage):
    print(distance, repr(node))

Nodes can be iterated depth-first and know their position in the tree:

In [ ]:
for child in outer_group.depth_first():
    print(repr(child), child.graph_order)

Booting the server is "just like"™ in `sclang`:

In [ ]:
server = supriya.Server()
server.boot()

We can print out to the interpreter a string representation of `scsynth`'s state:

In [ ]:
print(server)

Allocating a group recursively allocates its children. 

In [ ]:
outer_group.allocate()
print(server)

Let's visualize the allocated node structure:

In [ ]:
_ = supriya.graph(outer_group)

We can query synth controls and set them like a Python dictionary:

In [ ]:
synth_a["frequency"]

In [ ]:
synth_a["frequency"] = 654
print(synth_a["frequency"])

And we can set the controls of synths who are members of the subtree rooted at some group by setting a key of that the group like you would a dictionary:

In [ ]:
outer_group.controls["amplitude"] = 0.01
synth_a["amplitude"]

The synth controls are also explicitly modeled, hidden inside the "controls" interface:

In [ ]:
for control_name in synth_a.controls:
    print(repr(synth_a.controls[control_name]))

Groups also have a control interface, aggregating controls from synths in their subtree:

In [ ]:
for control_name in outer_group.controls:
    print(repr(outer_group.controls[control_name]))

We can allocate new nodes and move existing nodes in the same command

In [ ]:
synth_d = supriya.Synth(synthdef=supriya.assets.synthdefs.pad, name="D")
inner_group.extend([synth_d, synth_a])

Iteration continues to work:

In [ ]:
for node in outer_group.children:
    print(repr(node))

And the `Outer` group now knows about a new synth control name from the `pad` SynthDef:

In [ ]:
for control_name in outer_group.controls:
    print(control_name)

We can visualize the entire server node structure, including the root node and default group:

In [ ]:
_ = supriya.graph(server)

Let's free the `Outer` group:

In [ ]:
outer_group.free()

In [ ]:
print(server)

Explicitly freeing a group does not destructure its children:

In [ ]:
print(outer_group)

In [ ]:
_ = supriya.graph(outer_group)

In [ ]:
server.quit()

## OSC Command Aggregation

What's Supriya _actually_ doing when we move nodes in and out of the server tree?

In [ ]:
server = supriya.Server().reboot()
server.debug_request_names = True  # for legibility

Here's the same node structure as before...

In [ ]:
synth_a = supriya.Synth(frequency=333, name="A")
synth_b = supriya.Synth(frequency=444, name="B")
synth_c = supriya.Synth(frequency=555, name="C")
outer_group = supriya.Group(name="Outer")
inner_group = supriya.Group(name="Inner")
outer_group.extend([synth_a, inner_group, synth_c])
inner_group.append(synth_b)
print(outer_group)

Let's allocate the default SynthDef manually (why?):

In [ ]:
supriya.assets.synthdefs.default.allocate()

We can spy on OSC messages going to and coming from `scsynth`:

In [ ]:
with server.osc_io.capture() as transcript:
    outer_group.allocate()

In [ ]:
for timestamp, osc_message in transcript.sent_messages:
    print(repr(osc_message))

- What was sent when we allocated that group? An OSC bundle.
- What's an OSC bundle? A bunch of OSC messages meant to execute simultaneously.
- Supriya models OSC bundles and OSC messages explicitly as classes
- The OSC messages here are a linearized version of depth-first allocation of the nodes in the subtree.

We also have the responses from the server to each of those `/s_new` and `/g_new` commands.
And the `/synced` response as well...

In [ ]:
for timestamp, osc_message in transcript.received_messages:
    print(repr(osc_message))

Recall that I manually allocated the default synthdef earlier.

Let's make a new synth_d using a simple sine-wave synthdef.

Then let's allocate the new synth and also move synth 1001 into the inner group.

In [ ]:
synth_d = supriya.Synth(synthdef=supriya.assets.synthdefs.simple_sine)
with server.osc_io.capture() as transcript:
    inner_group.extend([synth_d, synth_a])

Supriya knows if SynthDefs have previously been allocated.

When allocating new synths it will generate an `/d_recv` and add any node allocation / movement / free commands as the completion message:

In [ ]:
for timestamp, osc_message in transcript.sent_messages:
    print(repr(osc_message))

In [ ]:
for timestamp, osc_message in transcript.received_messages:
    print(repr(osc_message))

## Requests and Responses

Let's reboot...

In [ ]:
server = supriya.Server().reboot()

Ok, this is almost the same as before, just simpler:

In [ ]:
synth_a = supriya.Synth(frequency=333, name="A")
synth_b = supriya.Synth(synthdef=supriya.assets.synthdefs.pad, frequency=444, name="B")
outer_group = supriya.Group(name="Outer")
inner_group = supriya.Group(name="Inner")
outer_group.extend([synth_a, inner_group])
inner_group.append(synth_b)

In [ ]:
with server.osc_io.capture() as transcript:
    outer_group.allocate()

In [ ]:
for timestamp, request in transcript.requests:
    print(request)

None of the above is OSC. It's all explicitly class-modeled.

Note that some of the `node_id` and `target_node_id` arguments are actually references to specific `Group` or `Synth` objects rather than just integers.

When communicating a request like this to the server, we don't necessarily know the IDs of the nodes until we start to communicate it.

What happen's when we "run" a request?
- Linearize the request (if necessary) into a series of requests.
- Apply each request _locally_, including allocating the ID of each request's node; the request classes implement any necessary logic for local application.
- If we want to block until the server processes the request, register an OSC callback using the requests's knowledge of what to expect
- Convert the request to OSC and send it
- If blocking, wait until we receive the expected response.

In [ ]:
for timestamp, response in transcript.responses:
    print(response)

There are a _lot_ of `Request` and `Response` classes...

In [ ]:
print(dir(supriya.commands))

In [ ]:
server.quit()

### Musings

The synthesis server is a state machine.

OSC commands are state transitions.

The local server state is a lossy model of the (complete) synthesis server state.

Request classes (can/should) encapsulate the logic for performing their effects on the local state.

## SynthDef Builders

SynthDefs are created via SynthDefBuilders, which act as "context managers":

In [ ]:
builder = supriya.SynthDefBuilder(amplitude=0, bus=0, frequency=440)

SynthDef parameters are accessed via dictionary lookup on the builder:

In [ ]:
with builder as builder:
    sine = supriya.ugens.SinOsc.ar(frequency=builder["frequency"])
    source = sine * builder["amplitude"]
    supriya.ugens.Out.ar(
        bus=builder["bus"],
        source=source,
    )

In [ ]:
synthdef = builder.build(name="simple_sine")
print(repr(synthdef))

In [ ]:
_ = supriya.graph(synthdef)

More ways of "viewing" a SynthDef:

In [ ]:
print(str(synthdef))

In [ ]:
synthdef.compile()

### A slightly more complex SynthDef

In [ ]:
builder = supriya.SynthDefBuilder(amplitude=1, bus=0, frequency=440, decay_time=5, coefficient=0.1)

In [ ]:
with builder:
    envelope = supriya.ugens.EnvGen.kr(
        envelope=supriya.synthdefs.Envelope.linen(
            attack_time=0,
            sustain_time=builder["decay_time"],
            release_time=0,
        ),
        done_action=supriya.DoneAction.FREE_SYNTH,
    )
    source = supriya.ugens.Pluck.ar(
        source=supriya.ugens.WhiteNoise.ar() * builder["amplitude"],
        trigger=supriya.ugens.Impulse.kr(frequency=0),
        maximum_delay_time=0.1,
        delay_time=1 / builder["frequency"],
        decay_time=builder["decay_time"],
        coefficient=builder["coefficient"],
    )
    supriya.ugens.Out.ar(bus=builder["bus"], source=[source, source])

In [ ]:
pluck_synthdef = builder.build()

In [ ]:
_ = supriya.graph(pluck_synthdef)

SynthDefs do not need to be named. Supriya uses hashing to generate unique names:

In [ ]:
pluck_synthdef.name, pluck_synthdef.anonymous_name, pluck_synthdef.actual_name

In [ ]:
server = supriya.Server().boot()
with server.osc_io.capture() as transcript:
    pluck_synthdef.play()

In [ ]:
for timestamp, request in transcript.requests:
    print(request)

### A swarm of fretless mandolins

In [ ]:
count = 50

with supriya.SynthDefBuilder() as builder:
    frequencies = supriya.ugens.SinOsc.kr(
        frequency=[supriya.ugens.Rand.ir(0.05, 0.2) for _ in range(count)],
        phase=[supriya.ugens.Rand.ir(0., 1.0) for _ in range(count)],
    ).range(1000, 3000)
    plucks = supriya.ugens.Pluck.ar(
        source=[supriya.ugens.WhiteNoise.ar() * 0.1 for _ in range(count)],
        trigger=[supriya.ugens.Impulse.kr(frequency=supriya.ugens.Rand.ir(10, 12)) for _ in range(count)],
        maximum_delay_time=1 / 100,
        delay_time=1 / frequencies,
        decay_time=2,
        coefficient=[supriya.ugens.Rand.ir(0.01, 0.2) for _ in range(count)],
    )
    pans = supriya.ugens.Pan2.ar(
        source=plucks,
        position=[supriya.ugens.Rand.ir(-1, 1) for _ in range(count)]
    )
    mix = supriya.ugens.Mix.multichannel(pans, 2)
    supriya.ugens.Out.ar(source=supriya.ugens.LeakDC.ar(source=mix))

In [ ]:
angry_mandolins = builder.build()

This SynthDef is too large to send over UDP, but Supriya does the right thing:

In [ ]:
len(angry_mandolins.compile())

In [ ]:
server = supriya.Server().reboot()
with server.osc_io.capture() as transcript:
    synth = angry_mandolins.play()

In [ ]:
for timestamp, request in transcript.requests:
    print(request)

In [ ]:
synth.free()

Let's parameterize those mandolins:

In [ ]:
def make_mandolin_synthdef(count=50):
    with supriya.SynthDefBuilder() as builder:
        frequencies = supriya.ugens.SinOsc.kr(
            frequency=[supriya.ugens.Rand.ir(0.05, 0.2) for _ in range(count)],
            phase=[supriya.ugens.Rand.ir(0., 1.0) for _ in range(count)],
        ).range(1000, 3000)
        plucks = supriya.ugens.Pluck.ar(
            source=[supriya.ugens.WhiteNoise.ar() * 0.1 for _ in range(count)],
            trigger=[supriya.ugens.Impulse.kr(frequency=supriya.ugens.Rand.ir(10, 12)) for _ in range(count)],
            maximum_delay_time=1 / 100,
            delay_time=1 / frequencies,
            decay_time=2,
            coefficient=[supriya.ugens.Rand.ir(0.01, 0.2) for _ in range(count)],
        )
        pans = supriya.ugens.Pan2.ar(
            source=plucks,
            position=[supriya.ugens.Rand.ir(-1, 1) for _ in range(count)]
        )
        mix = supriya.ugens.Mix.multichannel(pans, 2)
        supriya.ugens.Out.ar(source=supriya.ugens.LeakDC.ar(source=mix))
    return builder.build()

In [ ]:
server.reboot()

In [ ]:
synth_1 = make_mandolin_synthdef(count=1).play()

In [ ]:
synth_5 = make_mandolin_synthdef(count=5).play()

In [ ]:
synth_25 = make_mandolin_synthdef(count=25).play()

In [ ]:
server.quit()

## SynthDef Factories

Adding another level of abstraction and convention.

`Synth` --> `SynthDef` --> `SynthDefBuilder` --> `SynthDefFactory`

A double allpass delay chain with wiggly modulation:

In [ ]:
factory = supriya.synthdefs.SynthDefFactory()

In [ ]:
def signal_block(builder, source, state):
    iterations = state.get('iterations') or 2
    for _ in range(iterations):
        decay_time = supriya.ugens.LFDNoise3.kr(
            frequency=[1] * state["channel_count"],
        )
        delay_time = supriya.ugens.LFDNoise3.kr(
            frequency=[1] * state["channel_count"],
        )
        source = supriya.ugens.AllpassC.ar(
            decay_time=decay_time.range(0.05, 0.5),
            delay_time=delay_time.range(0.05, 1.0),
            source=source,
            maximum_delay_time=1.0,
            )
    return source

In [ ]:
factory = factory.with_input()
factory = factory.with_output()
factory = factory.with_signal_block(signal_block)

In [ ]:
synthdef = factory.build()

In [ ]:
_ = supriya.graph(synthdef)

Let's make that a quadruple allpass:

In [ ]:
synthdef = factory.build(iterations=4)
_ = supriya.graph(synthdef)

Or an octuple allpass:

In [ ]:
synthdef = factory.build(iterations=8)
_ = supriya.graph(synthdef)

A _stereo_ quadruple allpass delay:

In [ ]:
synthdef = factory.build(iterations=4, channel_count=2)
_ = supriya.graph(synthdef)

A double allpass delay with dry/wet mix:

In [ ]:
factory = factory.with_output(crossfaded=True)
synthdef = factory.build(iterations=2)
_ = supriya.graph(synthdef)

Where the dry/wet mix follows a durated hanning window:

In [ ]:
factory = factory.with_output(crossfaded=True, windowed=True)
synthdef = factory.build(iterations=2)
_ = supriya.graph(synthdef)

Now add another signal block with DC-kill and limiting:

In [ ]:
def signal_block_post(builder, source, state):
    source = supriya.ugens.LeakDC.ar(source=source)
    source = supriya.ugens.Limiter.ar(duration=0.01, source=source)
    return source

In [ ]:
factory = factory.with_signal_block(signal_block_post)
synthdef = factory.build(iterations=2)
_ = supriya.graph(synthdef)

And a feedback loop:

In [ ]:
factory = factory.with_feedback_loop()
synthdef = factory.build(iterations=2)
_ = supriya.graph(synthdef)

Where the feedback loop has its own signal block:

In [ ]:
def feedback_block(builder, source, state):
    multiplier = supriya.ugens.LFDNoise1.kr(frequency=10).range(0.1, 0.9)
    if len(source) > 1:
        source = supriya.synthdefs.UGenArray((source[-1],) + source[:-1])
    return -(multiplier * source)

In [ ]:
factory = factory.with_feedback_loop(feedback_block)
synthdef = factory.build(iterations=2)
_ = supriya.graph(synthdef)

And a random seed ID:

In [ ]:
factory = factory.with_rand_id(1)
synthdef = factory.build(iterations=2)
_ = supriya.graph(synthdef)

And finally in stereo with four delays:

In [ ]:
synthdef = factory.build(iterations=4, channel_count=2)
_ = supriya.graph(synthdef)

## Non-realtime Session Model

Supriya has a parallel object-model for non-realtime composition.

In [ ]:
%reload_ext supriya.ext.ipython
import supriya

In [ ]:
session = supriya.nonrealtime.Session(2, 2)

with session.at(0):
    synth_a = session.add_synth(duration=6, frequency=444)
    group = session.add_group(add_action="ADD_TO_TAIL")
    
with session.at(2):
    synth_b = session.add_synth(duration=6, frequency=555, add_action="ADD_TO_TAIL")
    
with session.at(4):
    group.move_node(synth_a)

In [ ]:
_ = supriya.play(session)

In [ ]:
print(session.to_strings())

Just like the realtime model, nodes know about their parents and children.

With the caveat that you always have to ask "when...":

In [ ]:
with session.at(1):
    print(synth_a.get_parentage())

In [ ]:
with session.at(5):
    print(synth_a.get_parentage())

In [ ]:
with session.at(1):
    print(session.root_node.get_children())

In [ ]:
with session.at(5):
    print(session.root_node.get_children())

In [ ]:
for osc_bundle in session.to_osc_bundles(duration=10):
    for osc_message in osc_bundle.contents:
        print(osc_bundle.timestamp, repr(osc_message))

A slightly trickier example:

In [ ]:
session = supriya.nonrealtime.Session(2, 2)

with session.at(0):
    outer_group = session.add_group()
    synth_a = outer_group.add_synth(duration=6, frequency=333)
    inner_group = outer_group.add_group(add_action="ADD_TO_TAIL")
    
with session.at(2):
    synth_a["frequency"] = 444
    
with session.at(4):
    synth_b = inner_group.add_synth(duration=4, frequency=555)
    
with session.at(6):
    synth_b.move_node(synth_a, "ADD_AFTER")

In [ ]:
_ = supriya.play(session, duration=10)

Nodes know about their parameters through time:

In [ ]:
with session.at(1):
    print(synth_a["frequency"])

In [ ]:
with session.at(3):
    print(synth_a["frequency"])

In [ ]:
print(session.to_strings())

We can perform some more exotic operations on the nodes, like splitting a group but keeping its children intact:

In [ ]:
with session.at(7.5):
    outer_group.split(split_occupiers=False)

In [ ]:
_ = supriya.play(session, duration=10)

Group 1000 has become group 1004:

In [ ]:
print(session.to_strings())

Look at the OSC messages at timestamp 7.5:

In [ ]:
for osc_bundle in session.to_osc_bundles(duration=40):
    for osc_message in osc_bundle.contents:
        print(osc_bundle.timestamp, repr(osc_message))

### The `__render__` protocol

Supriya's NRT knows about soundfiles and _things that could, in the future, be soundfiles_.

This is expressed via the `__render__` protocol. If a class implements a `__render__` method that writes audio to disk and returns the path, then instances of that class can be used anywhere you would pass the path to a soundfile.

In [ ]:
%load_ext supriya.ext.ipython
import supriya

In [ ]:
say_supriya = supriya.Say("Speak and Spell Supriya", voice="Daniel")
_ = supriya.play(say_supriya)

### NRT Dependency Tree (turtles all the way down)

Let's make a quick mono-to-stereo sampler player:

In [ ]:
with supriya.SynthDefBuilder(buffer_id=0, gain=0, pan=0, rate=1) as builder:
    rate_scale = supriya.ugens.BufRateScale.kr(buffer_id=builder["buffer_id"])
    source = supriya.ugens.PlayBuf.ar(
        buffer_id=builder["buffer_id"],
        channel_count=1,
        done_action=supriya.DoneAction.FREE_SYNTH,
        rate=builder["rate"] * rate_scale,
    ) * builder["gain"].db_to_amplitude()
    source = supriya.ugens.Pan2.ar(source=source, position=builder["pan"])
    supriya.ugens.Out.ar(bus=0, source=source)
    
mono_to_stereo_sampler_synthdef = builder.build("mono_to_stereo_sampler")

Add the `Say` instance as a buffer, then play it fives times:

In [ ]:
say_supriya_session = supriya.Session(2, 2)

with say_supriya_session.at(0):
    buffer_ = say_supriya_session.add_buffer(
        channel_count=1,
        file_path=say_supriya,
    )
    
for i in range(5):
    with say_supriya_session.at(i):
        synth = say_supriya_session.add_synth(
            synthdef=mono_to_stereo_sampler_synthdef,
            buffer_id=buffer_,
            rate=2 ** (-i / 4),
            pan=(i - 2) / 2,
        )

In [ ]:
_ = supriya.play(say_supriya_session, duration=10, print_transcript=True)

Now let's make a quick stereo-to-stereo sample player:

In [ ]:
with supriya.SynthDefBuilder(buffer_id=0, gain=0, pan=0, rate=1) as builder:
    rate_scale = supriya.ugens.BufRateScale.kr(buffer_id=builder["buffer_id"])
    source = supriya.ugens.PlayBuf.ar(
        buffer_id=builder["buffer_id"],
        channel_count=2,
        done_action=supriya.DoneAction.FREE_SYNTH,
        rate=builder["rate"] * rate_scale,
    ) * builder["gain"].db_to_amplitude()
    supriya.ugens.Out.ar(bus=0, source=source)
    
stereo_to_stereo_sampler_synthdef = builder.build("stereo_to_stereo_sampler")

The previous session is now used as the file path for the buffer in this new session:

In [ ]:
say_supriya_session_session = supriya.Session(2, 2)

with say_supriya_session_session.at(0):
    buffer_ = say_supriya_session_session.add_buffer(
        channel_count=2,
        file_path=say_supriya_session,
    )
    
for i in range(4):
    with say_supriya_session_session.at(i):
        synth = say_supriya_session_session.add_synth(
            synthdef=stereo_to_stereo_sampler_synthdef,
            buffer_id=buffer_,
            rate=2 ** (-i / 4)
        )

In [ ]:
_ = supriya.play(say_supriya_session_session, duration=12, print_transcript=True)

Deeper and deeper:

In [ ]:
allpass_panic_synthdef = (
    factory
    .with_input(private=True)  # separate in and out bus IDs
    .with_output(crossfaded=True)
    .build(name="allpass_panic", channel_count=2, iterations=8)
)

meta_session = supriya.Session(2, 2, input_=say_supriya_session_session)

with meta_session.at(0):
    # we need to read from input bus and write to the output bus
    meta_session.add_synth(
        synthdef=allpass_panic_synthdef,
        crossfade=0.5,
        in_=meta_session.audio_input_bus_group,  # input buses are object-modeled
    )
    meta_session.set_rand_seed(rand_id=0, rand_seed=1)

In [ ]:
for osc_bundle in meta_session.to_osc_bundles(duration=40):
    for osc_message in osc_bundle.contents:
        print(osc_bundle.timestamp, repr(osc_message))

In [ ]:
_ = supriya.play(meta_session, duration=30, print_transcript=True)

In [ ]:
server.quit()
for path in supriya.output_path.iterdir():
    path.unlink()